In [1]:
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path
from copy import deepcopy
from tqdm import tqdm

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
random.seed(42)

In [3]:
def calc_accuracy(y_true, y_pred, metric='mae'):
    if metric == 'mae':
        acc = mean_absolute_error(y_true, y_pred)
    elif metric == 'r2':
        acc = r2_score(y_true, y_pred)
    elif metric == 'rmse':
        acc = root_mean_squared_error(y_true, y_pred)

    return acc

## Consensus methods

In [4]:
from consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor
from sklearn.linear_model import LinearRegression, Ridge

In [5]:
method_list = [(SystematicSearchRegressor(cons_size=1, metric='rmse'), 'Best'),
               (RandomSearchRegressor(cons_size=10, n_iter=10000, metric='rmse'), 'Random'),
               (SystematicSearchRegressor(cons_size=10, metric='rmse'), 'Systematic'),
               (GeneticSearchRegressor(cons_size=10, metric='rmse'), 'Genetic')]

## Consensus comparison
### Input/output data path

In [6]:
bench_folder =  Path("benchmark_model_prediction/acf").resolve()

### Consensus building

In [7]:
res_df_val = pd.DataFrame()
res_df_test = pd.DataFrame()

for bench_name in tqdm(os.listdir(bench_folder)):

    # load data
    df_val = pd.read_csv(os.path.join(bench_folder, bench_name, f"{bench_name}_val.csv"))
    df_test = pd.read_csv(os.path.join(bench_folder, bench_name, f"{bench_name}_test.csv"))

    # remove y_true column prof predictions table
    x_val, y_val = df_val.iloc[:, 1:], df_val.iloc[:, 0]
    x_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]
    
    # build consensus
    for method_func, method_name in method_list:
        cons = method_func.run(x_val, y_val)
        y_pred = x_val[cons].mean(axis=1)
        #
        res_df_val.loc[bench_name, method_name] = calc_accuracy(y_val, x_val[cons].mean(axis=1), metric='r2')
        res_df_test.loc[bench_name, method_name] = calc_accuracy(y_test, x_test[cons].mean(axis=1), metric='r2')
        
    # stacking model
    stacking_model = LinearRegression()
    stacking_model.fit(x_val, y_val)
    #
    res_df_val.loc[bench_name, "Stacking"] = calc_accuracy(y_val, stacking_model.predict(x_val), metric='r2')
    res_df_test.loc[bench_name, "Stacking"] = calc_accuracy(y_test, stacking_model.predict(x_test), metric='r2')

100%|███████████████████████████████████████████████████████████████████████████████████| 43/43 [08:59<00:00, 12.56s/it]


In [8]:
# res_df_val.round(2)

In [9]:
# res_df_test.round(2)

## Meta statistics

In [10]:
from collections import Counter

### 1. Mean accuracy

In [11]:
res_df_val.mean(axis=0).round(2)

Best          0.65
Random        0.63
Systematic    0.66
Genetic       0.71
Stacking      0.94
dtype: float64

In [12]:
res_df_test.mean(axis=0).round(2)

Best          0.57
Random        0.60
Systematic    0.61
Genetic       0.62
Stacking     -3.60
dtype: float64

### 2. Top-N stat

In [13]:
tmp = []
for dataset in res_df_test.index:
    min_i = res_df_test.loc[dataset].argmin()
    max_i = res_df_test.loc[dataset].argmax()
    #
    tmp.append(res_df_test.loc[dataset].index[max_i])
#
Counter(tmp)

Counter({'Genetic': 24, 'Random': 9, 'Systematic': 8, 'Best': 2})